# Imports

In [1]:
#!pip install -U scikit-learn

In [2]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from torch import nn
from numpy import mean
#!pip install igraph
from igraph import Graph as igraphGraph

# Input

In [3]:
transformation = Transformation()

user_number_triangles = 500
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [4]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [5]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [6]:
class DevConv(nn.Module):
    def __init__(self, graph, output_dimension):
        super().__init__()
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = torch.tensor(np.random.random((output_dimension))) #change #passer en tensor
        self.W_theta = torch.tensor(np.array([0.1, 0.1, 0.1]).reshape(1, -1))  # change #passer en tensor
        print("self.W_theta.shape : " )
        print(self.W_theta.shape)
    
    # objective plus de np dans le forward, que utiliser des fonction pytorch (nn.)
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        print("previous_inclusion score : ")
        print(previous_inclusion_score.shape)
        
        print("list_inc_score : ")
        print(list_inc_score.shape)
        result_np = torch.stack([previous_inclusion_score, torch.tensor(list_inc_score)])
        print("result_np.shape : ")
        print(result_np.shape)
        
        result_np = torch.mean(result_np, axis=0)
        print("mean.shape : ")
        print(result_np.shape)
        
        # result_np =  torch.from_numpy(result_np)
        # print("devconv tensor shape : ")
        # print(result_np.shape)
        #return torch.from_numpy(np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0))
        return result_np
    
    
#Tensor.numpy() pour commencer, à la fin repasser en tensor normaux.
        

In [7]:
class GNN_Model(nn.Module):
    def __init__(self):
        super(GNN_Model, self).__init__()
        # self.linear = nn.Sequential(
        #     DevConv(graph, 1),
        #     nn.ReLU(),
        #     DevConv(graph, 64),
        #     nn.ReLU(),
        #     DevConv(graph, 1),
        #     nn.Sigmoid()
        # )
        self.devconv = DevConv(graph,1)
        self.relu = nn.ReLU()
        self.devconv2 = DevConv(graph, 64)
        self.relu2 = nn.ReLU()
        self.devconv3 = DevConv(graph,1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x= self.devconv(x)
        x= torch.tensor(x)
        #print(x)
        x= self.relu(x)
        x= self.devconv2(x)
        x= self.relu2(x)
        x= self.devconv3(x)
        x= self.sigmoid(x)
        return x

In [8]:
torch.manual_seed(0)  #  for repeatable results
gnn = GNN_Model()
# inp = np.array([[[[1,2,3,4],  # batch(=1) x channels(=1) x height x width
#                   [1,2,3,4],
#                   [1,2,3,4]]]])
inp = np.array([])
x = torch.tensor(inp, dtype=torch.float)
inclusion_score = gnn(x)

self.W_theta.shape : 
torch.Size([1, 3])
self.W_theta.shape : 
torch.Size([1, 3])
self.W_theta.shape : 
torch.Size([1, 3])


previous_inclusion score : 
torch.Size([5999])
list_inc_score : 
(5999,)
result_np.shape : 
torch.Size([2, 5999])
mean.shape : 
torch.Size([5999])
previous_inclusion score : 
torch.Size([5999])
list_inc_score : 
(5999,)
result_np.shape : 
torch.Size([2, 5999])
mean.shape : 
torch.Size([5999])


In [9]:
# devconv = DevConv(graph, 1)
# inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
# inclusion_score = relu(inclusion_score)
# print(inclusion_score)
# print("inclusion_score shape :")
# print(inclusion_score.shape)

# devconv = DevConv(graph, 64)
# inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
# inclusion_score = relu(inclusion_score)
# print(inclusion_score)
# print("inclusion_score shape :")
# print(inclusion_score.shape)

# devconv = DevConv(graph, 1)
# inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
# inclusion_score = sigmoid(inclusion_score)
# print(inclusion_score)
# print(inclusion_score.shape)

### Multinomial Sampling

In [10]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

TypeError: sum() received an invalid combination of arguments - got (axis=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: axis, out
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [ ]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(list(graph._node.keys()))[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[85.318504 ,  5.7970243, 92.65986  ],
       [85.90413  ,  5.7970243, 92.104126 ],
       [84.064095 ,  5.7970243, 93.67566  ],
       [83.39875  ,  5.7970243, 94.13294  ],
       [87.47566  ,  5.7970243, 90.2641   ]], dtype=float32)

# KNN extended graph

In [ ]:
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11567


# Edge Predictor

In [ ]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

self.W_theta.shape : 
torch.Size([1, 3])


(1500, 64)

In [ ]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [ ]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [ ]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [ ]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  (1, 8)	1
  (1, 9)	1
  (1, 12)	1
  :	:
  (1498, 1443)	1
  (1498, 1468)	1
  (1498, 1469)	1
  (1498, 1481)	1
  (1498, 1486)	1
  (1498, 1493)	1
  (1498, 1494)	1
  (1498, 1495)	1
  (1498, 1496)	1
  (1498, 1497)	1
  (1498, 1499)	1
  (1499, 1432)	1
  (1499, 1433)	1
  (1499, 1434)	1
  (1499, 1439)	1
  (1499, 1443)	1
  (1499, 1468)	1
  (1499, 1481)	1
  (1499, 1486)	1
  (1499, 1493)	1
  (1499, 1494)	1
  (1499, 1495)	1
  (1499, 1496)	1
  (1499, 1497)	1
  (1499, 1498)	1
[[0.066535   0.06651779 0.06654316 ... 0.06571079 0.06570608 0.06569831]
 [1.18436574 0.07210451 0.07199037 ... 0.07554197 0.07556105 0.07559255]
 [1.08724829 0.50002084 0.06619418 ... 0.06323582 0.06322007 0.06319407]
 ...
 [1.96554016 1.83970835 1.97302663 ... 0.0583066  0.05824423 0.

In [ ]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [ ]:
print(A_s)  # symmétrique
print(A_s.shape)

[[4.21275686e-07 4.68134104e-07 4.21238709e-07 ... 1.15218045e-04
  1.17097626e-04 1.19433965e-04]
 [4.68134104e-07 5.20338055e-07 4.69798077e-07 ... 1.31456383e-04
  1.33615608e-04 1.36306884e-04]
 [4.21238709e-07 4.69798077e-07 4.24966061e-07 ... 1.11595096e-04
  1.13405424e-04 1.15652511e-04]
 ...
 [1.15218045e-04 1.31456383e-04 1.11595096e-04 ... 1.05795152e-01
  1.07810931e-01 1.10792495e-01]
 [1.17097626e-04 1.33615608e-04 1.13405424e-04 ... 1.07810931e-01
  1.09865996e-01 1.12906390e-01]
 [1.19433965e-04 1.36306884e-04 1.15652511e-04 ... 1.10792495e-01
  1.12906390e-01 1.16071338e-01]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [ ]:
nodes = list(extended_graph.nodes())
edges = list(extended_graph.edges())

igraph_g = igraphGraph(directed=extended_graph.is_directed())

igraph_g.add_vertices(nodes)
igraph_g.add_edges([(nodes.index(u), nodes.index(v)) for u, v in edges])
print(igraph_g.summary())

IGRAPH UN-- 1500 11567 -- 
+ attr: name (v)


In [ ]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[ 798  845 1006]
 [ 567  573  845]
 [1469 1471 1472]]
31039


In [ ]:
triangles = np.array(igraph_g.vs['name'])[triangles_ids_igraph]
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[ 75.          -6.5098743    0.92127824]
  [ 77.63298     -6.9390492    1.667854  ]
  [ 75.          -5.0095077    2.234973  ]]

 [[ 81.81048     -8.760632    -3.010477  ]
  [ 80.11056     -8.722893    -1.8758512 ]
  [ 77.63298     -6.9390492    1.667854  ]]

 [[ 39.          -1.1623888  -13.722893  ]
  [ 39.           0.16653232 -13.798424  ]
  [ 39.           0.8314692  -13.760632  ]]]
31039


In [ ]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(igraph_g.vs))
p_init[i] = (A_s_ij + A_s_ik + A_s_jk) / 3

#### Calculate barycenter

In [ ]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(31039, 3)
[[ 7.5877663e+01 -6.1528106e+00  1.6080351e+00]
 [ 7.9851341e+01 -8.1408577e+00 -1.0728248e+00]
 [ 3.9000000e+01 -5.4795761e-02 -1.3760650e+01]]


#### KNN Tri

In [ ]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [ ]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(31039, 19, 3)


#### calculate e norm matrix

In [ ]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(31039, 3)
[[2.770226  1.9942151 3.313207 ]
 [2.0441453 6.531189  4.677428 ]]


#### Calculate r

In [ ]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(31039,)

In [ ]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(31039, 19)
(31039, 19)


In [ ]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(31039, 19, 5)


#### Calculate f

In [ ]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999999998
(1500,)
[0.00065867 0.00065867 0.00065867 ... 0.00073563 0.00065867 0.00065867]


C:\Users\basti\AppData\Local\Temp\ipykernel_17160\1047055924.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))


# Simplified Mesh

In [ ]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[:1])

(500, 3, 3)
[[[80.28243   -2.9401166 97.457664 ]
  [80.20638   -3.5589852 97.223595 ]
  [79.285     -4.1574683 97.191864 ]]]


In [ ]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 440
Number of edges: 907


In [ ]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)